In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For this assignment, we need to load in the following modules
import requests
import StringIO
import zipfile
import scipy.stats 

In [4]:
Todo Using the movie-lens 1M data and pandas.read_table read in all three files (users, ratings, movies) into pandas DataFrames. I recommend giving columns names directly to read_table for each case.
HINT: you will need to open the files in a text editor to see what the separator character. Also, the columns for each file correspond to the following:
users.dat: user id, gender, age, occupation code, zip
ratings.dat: user id, movie id, rating, timestamp
movies.dat: movie id, title, genre
Use the data combining tools discussed above to combine these three objects into a single object named movie_data

SyntaxError: invalid syntax (<ipython-input-4-1ba9a61b7c33>, line 1)

In [2]:
# Create function for downloading the zip file
def getZIP(zipFileName):
    r = requests.get(zipFileName).content
    s = StringIO.StringIO(r)
    zf = zipfile.ZipFile(s, 'r') # Read in a list of zipped files
    return zf

In [4]:
url = 'http://www.grouplens.org/system/files/ml-1m.zip'
zf = getZIP(url)
print zf.namelist()

['ml-1m/', 'ml-1m/movies.dat', 'ml-1m/ratings.dat', 'ml-1m/README', 'ml-1m/users.dat']


In [68]:
tablenames = zf.namelist()
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_table(zf.open(tablenames[tablenames.index('ml-1m/movies.dat')]),sep='::',names=m_cols)
print "Number of rows: %i" % movies.shape[0]
movies.head()

Number of rows: 3883


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [69]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_table(zf.open(tablenames[tablenames.index('ml-1m/ratings.dat')]),sep='::',names=r_cols)
print "Number of rows: %i" % ratings.shape[0]
ratings.head()

Number of rows: 1000209


,user_id,movie_id,rating,unix_timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [25]:
u_cols = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users = pd.read_table(zf.open(tablenames[tablenames.index('ml-1m/users.dat')]),sep='::',names=u_cols)
print "Number of rows: %i" % users.shape[0]
users.head()

Number of rows: 6040


,user_id,sex,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [26]:
# merging the 3 datasets

mov_rat=pd.merge(movies, ratings, how="inner", on=['movie_id'])
print "Number of rows: %i" % mov_rat.shape[0]
mov_rat.head()

Number of rows: 1000209


,movie_id,title,genre,user_id,rating,unix_timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [27]:
# merging the 3 datasets

mov_rat_user=pd.merge(mov_rat, users, how="inner", on=['user_id'])
print "Number of rows: %i" % mov_rat_user.shape[0]
mov_rat_user.head()

Number of rows: 1000209


,movie_id,title,genre,user_id,rating,unix_timestamp,sex,age,occupation,zip_code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [141]:
# alternative way of merging 3 datasets in panda

mov_rat_usr=pd.merge(pd.merge(movies,ratings, how="inner",on='movie_id'),users, how="inner",on='user_id')
print "Number of rows: %i" % mov_rat_usr.shape[0]
mov_rat_usr.head()

Number of rows: 1000209


,movie_id,title,genre,user_id,rating,unix_timestamp,sex,age,occupation,zip_code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [39]:
Todo Use the movie_data object from the previous exercise and compute the following things:
The 5 movies with the most number of ratings
A new object called active_titles that is made up of movies each having at least 250 ratings
For the subset of movies in the active_titles list compute the following:
The 3 movies with the highest average rating for females. Do the same for males.
The 10 movies men liked much more than women and the 10 movies women liked more than men (use the difference in average ratings and sort ascending and descending).
The 5 movies that had the highest standard deviation in rating.
HINT: For some of these operations it might be helpful to compute a pivot table using the pivot_table method of a DataFrame.
NOTE: You might also want to do some analysis on the movie genre. To do this you would have to be comfortable with using pandas vectorized string methods and probably some regular expressions. Those topics are a bit bigger than what we can cover here, but if you are looking for more practice, they are a good place to start.

SyntaxError: invalid syntax (<ipython-input-39-218cf9b31d4e>, line 1)

In [118]:
# The 5 movies with the most number of ratings
mov_rat_usr.drop_duplicates(['movie_id','title']).sort('rating',ascending = False).head(5)

,movie_id,title,genre,user_id,rating,unix_timestamp,sex,age,occupation,zip_code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
883,1275,Highlander (1986),Action|Adventure,18,5,978153241,F,18,3,95825
14423,1169,American Dream (1990),Documentary,182,5,977085616,M,18,4,03052
14087,3844,Steel Magnolias (1989),Drama,175,5,977112694,F,25,2,95123
14050,3145,"Cradle Will Rock, The (1999)",Drama,175,5,977113034,F,25,2,95123


In [119]:
# A new object called active_titles that is made up of movies each having at least 250 ratings

mov_rat_usr['active_titles'] = mov_rat_usr['rating']>= 250
mov_rat_usr.head()

,movie_id,title,genre,user_id,rating,unix_timestamp,sex,age,occupation,zip_code,active_titles
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,False
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067,False
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,False
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,False
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067,False


In [143]:
# For the subset of movies in the active_titles list compute the following:
# The 3 movies with the highest average rating for females. Do the same for males.
movF=mov_rat_usr[mov_rat_usr.sex == 'F']
movF.head()

,movie_id,title,genre,user_id,rating,unix_timestamp,sex,age,occupation,zip_code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [172]:
movie_stats = pd.DataFrame(movF.groupby('title')['rating'].mean()).sort('rating',ascending=False)
movie_stats.head()


,rating
title,
Clean Slate (Coup de Torchon) (1981),5
"Ballad of Narayama, The (Narayama Bushiko) (1958)",5
Raw Deal (1948),5
Bittersweet Motel (2000),5
Skipped Parts (2000),5


In [174]:
# The 10 movies men liked much more than women and the 10 movies women liked more than 
pivoted = mov_rat_usr.pivot_table(index=['movie_id', 'title'],
                           columns=['sex'],
                           values='rating',
                           fill_value=0)
pivoted.head()

,sex,F,M
movie_id,title,,
1,Toy Story (1995),4.187817,4.130552
2,Jumanji (1995),3.278409,3.175238
3,Grumpier Old Men (1995),3.073529,2.994152
4,Waiting to Exhale (1995),2.976471,2.482353
5,Father of the Bride Part II (1995),3.212963,2.888298


In [176]:
# difference between F and M

pivoted['diff']=pivoted['F']-pivoted['M']
pivoted.head()

,sex,F,M,diff
movie_id,title,,,
1,Toy Story (1995),4.187817,4.130552,0.057265
2,Jumanji (1995),3.278409,3.175238,0.103171
3,Grumpier Old Men (1995),3.073529,2.994152,0.079377
4,Waiting to Exhale (1995),2.976471,2.482353,0.494118
5,Father of the Bride Part II (1995),3.212963,2.888298,0.324665


In [178]:
pivoted.sort('diff',ascending=False)[:10]

,sex,F,M,diff
movie_id,title,,,
3881,Bittersweet Motel (2000),5.0,0,5.0
3382,Song of Freedom (1936),5.0,0,5.0
3607,One Little Indian (1973),5.0,0,5.0
2309,"Inheritors, The (Die Siebtelbauern) (1998)",4.5,0,4.5
1832,Heaven's Burning (1997),4.0,0,4.0
2438,Outside Ozona (1998),4.0,0,4.0
398,Frank and Ollie (1995),4.0,0,4.0
396,Fall Time (1995),4.0,0,4.0
624,Condition Red (1995),4.0,0,4.0


In [180]:
pivoted.sort('diff',ascending=True)[:10]

,sex,F,M,diff
movie_id,title,,,
3172,Ulysses (Ulisse) (1954),0,5.0,-5.0
989,Schlafes Bruder (Brother of Sleep) (1995),0,5.0,-5.0
3280,"Baby, The (1973)",0,5.0,-5.0
1830,Follow the Bitch (1998),0,5.0,-5.0
3656,Lured (1947),0,5.0,-5.0
3233,Smashing Time (1967),0,5.0,-5.0
2480,Dry Cleaning (Nettoyage � sec) (1997),0,4.5,-4.5
1795,"Callej�n de los milagros, El (1995)",0,4.5,-4.5
557,Mamma Roma (1962),0,4.5,-4.5


In [183]:
# The 5 movies that had the highest standard deviation in rating

movStd = pd.DataFrame(mov_rat_usr.groupby('title')['rating'].std()).sort('rating',ascending=False)
movStd.head()

,rating
title,
Foreign Student (1994),2.828427
Criminal Lovers (Les Amants Criminels) (1999),2.309401
Identification of a Woman (Identificazione di una donna) (1982),2.121320
Tokyo Fist (1995),2.121320
Better Living (1998),2.121320


In [184]:
url = 'http://www.ssa.gov/oact/babynames/names.zip'
zf = getZIP(url)
print zf.namelist()

['yob1881.txt', 'yob1882.txt', 'yob1883.txt', 'yob1884.txt', 'yob1885.txt', 'yob1886.txt', 'yob1887.txt', 'yob1888.txt', 'yob1889.txt', 'yob1890.txt', 'yob1891.txt', 'yob1892.txt', 'yob1893.txt', 'yob1894.txt', 'yob1895.txt', 'yob1896.txt', 'yob1897.txt', 'yob1898.txt', 'yob1899.txt', 'yob1900.txt', 'yob1901.txt', 'yob1902.txt', 'yob1903.txt', 'yob1904.txt', 'yob1905.txt', 'yob1906.txt', 'yob1907.txt', 'yob1908.txt', 'yob1909.txt', 'yob1910.txt', 'yob1911.txt', 'yob1912.txt', 'yob1913.txt', 'yob1914.txt', 'yob1915.txt', 'yob1916.txt', 'yob1917.txt', 'yob1918.txt', 'yob1919.txt', 'yob1920.txt', 'yob1921.txt', 'yob1922.txt', 'yob1923.txt', 'yob1924.txt', 'yob1925.txt', 'yob1926.txt', 'yob1927.txt', 'yob1928.txt', 'yob1929.txt', 'yob1930.txt', 'yob1931.txt', 'yob1932.txt', 'yob1933.txt', 'yob1934.txt', 'yob1935.txt', 'yob1936.txt', 'yob1937.txt', 'yob1938.txt', 'yob1939.txt', 'yob1940.txt', 'yob1941.txt', 'yob1942.txt', 'yob1943.txt', 'yob1944.txt', 'yob1945.txt', 'yob1946.txt', 'yob1947.

In [192]:
tablenames = zf.namelist()
m_cols = ['name', 'gender', 'socialID']
movies = pd.read_table(zf.open(tablenames[tablenames.index('yob1881.txt')]),sep=',',names=m_cols)
print "Number of rows: %i" % movies.shape[0]
movies.head()


Number of rows: 1935


In [200]:
file_list = zf.namelist()
file_list

['yob1881.txt',
 'yob1882.txt',
 'yob1883.txt',
 'yob1884.txt',
 'yob1885.txt',
 'yob1886.txt',
 'yob1887.txt',
 'yob1888.txt',
 'yob1889.txt',
 'yob1890.txt',
 'yob1891.txt',
 'yob1892.txt',
 'yob1893.txt',
 'yob1894.txt',
 'yob1895.txt',
 'yob1896.txt',
 'yob1897.txt',
 'yob1898.txt',
 'yob1899.txt',
 'yob1900.txt',
 'yob1901.txt',
 'yob1902.txt',
 'yob1903.txt',
 'yob1904.txt',
 'yob1905.txt',
 'yob1906.txt',
 'yob1907.txt',
 'yob1908.txt',
 'yob1909.txt',
 'yob1910.txt',
 'yob1911.txt',
 'yob1912.txt',
 'yob1913.txt',
 'yob1914.txt',
 'yob1915.txt',
 'yob1916.txt',
 'yob1917.txt',
 'yob1918.txt',
 'yob1919.txt',
 'yob1920.txt',
 'yob1921.txt',
 'yob1922.txt',
 'yob1923.txt',
 'yob1924.txt',
 'yob1925.txt',
 'yob1926.txt',
 'yob1927.txt',
 'yob1928.txt',
 'yob1929.txt',
 'yob1930.txt',
 'yob1931.txt',
 'yob1932.txt',
 'yob1933.txt',
 'yob1934.txt',
 'yob1935.txt',
 'yob1936.txt',
 'yob1937.txt',
 'yob1938.txt',
 'yob1939.txt',
 'yob1940.txt',
 'yob1941.txt',
 'yob1942.txt',
 'yob194

In [204]:
from glob import glob

In [211]:

files = glob('*.TXT')
data = []
for file_path in files:
    data.append(
        np.genfromtxt(file_path,delimiter=',',names=m_cols))

In [61]:
mov_rat_usr.title.value_counts()[:25]
movie_stats = mov_rat_usr.groupby('title').agg({'rating': [np.size, np.mean]}).sort(ascending=False)
movie_stats.head()

rating          
                                             size      mean
title                                                      
eXistenZ (1999)                               410  3.256098
Zeus and Roxanne (1997)                        23  2.521739
Zero Kelvin (Kj�rlighetens kj�tere) (1995)      2  3.500000
Zero Effect (1998)                            301  3.750831
Zed & Two Noughts, A (1985)                    29  3.413793

In [66]:
movie_stats.sort([('rating','mean')], ascending=False).head()

rating     
                                     size mean
title                                         
Gate of Heavenly Peace, The (1995)      3    5
Lured (1947)                            1    5
Ulysses (Ulisse) (1954)                 1    5
Smashing Time (1967)                    2    5
Follow the Bitch (1998)                 1    5

In [70]:
atleast_100 = movie_stats['rating']['size'] >= 250
movie_stats[atleast_100].sort([('rating', 'mean')], ascending=False)[:15]

rating          
                                                     size      mean
title                                                              
Seven Samurai (The Magnificent Seven) (Shichini...    628  4.560510
Shawshank Redemption, The (1994)                     2227  4.554558
Godfather, The (1972)                                2223  4.524966
Close Shave, A (1995)                                 657  4.520548
Usual Suspects, The (1995)                           1783  4.517106
Schindler's List (1993)                              2304  4.510417
Wrong Trousers, The (1993)                            882  4.507937
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)         470  4.491489
Raiders of the Lost Ark (1981)                       2514  4.477725
Rear Window (1954)                                   1050  4.476190
Star Wars: Episode IV - A New Hope (1977)            2991  4.453694
Third Man, The (1949)                                 480  4.452083
Dr. Strangelove or: How I Learned to Stop Worry...   1367  4.449890
Wallace & Gromit: The Best of Aardman Animation...    438  4.426941
To Kill a Mockingbird (1962)                          928  4.425647

In [92]:
most_50 = mov_rat_usr.groupby('movie_id').size()[:50]
most_50.head()

movie_id
1    2077
2     701
3     478
4     170
5     296
dtype: int64

In [94]:
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
mov_rat_usr['age_group'] = pd.cut(mov_rat_usr.age, range(0, 81, 10), right=False, labels=labels)
mov_rat_usr[['age', 'age_group']].drop_duplicates()[:10]

,age,age_group
0,1,0-9
53,50,50-59
124,25,20-29
369,35,30-39
770,18,10-19
2778,45,40-49
5001,56,50-59


In [96]:
mov_rat_usr.groupby('age_group').agg({'rating': [np.size, np.mean]})

rating          
             size      mean
age_group                  
0-9         27211  3.549520
10-19      183536  3.507573
20-29      395556  3.545235
30-39      199003  3.618162
40-49       83633  3.638062
50-59      111270  3.732677
60-69         NaN       NaN
70-79         NaN       NaN

In [99]:
mov_rat_usr.set_index('movie_id', inplace=True)
by_age = mov_rat_usr.loc[most_50.index].groupby(['title', 'age_group'])
by_age.rating.mean().head(15)


title                                  age_group
Ace Ventura: When Nature Calls (1995)  0-9          2.666667
                                       10-19        2.594595
                                       20-29        2.510345
                                       30-39        2.428571
                                       40-49        2.318182
                                       50-59        2.058824
Across the Sea of Time (1995)          10-19        2.500000
                                       20-29        3.333333
                                       30-39        4.000000
                                       50-59        5.000000
American President, The (1995)         0-9          3.476190
                                       10-19        3.693989
                                       20-29        3.734848
                                       30-39        3.848341
                                       40-49        3.988889
Name: rating, dtype: float64

In [101]:
by_age.rating.mean().unstack(1).fillna(0)[10:20]

age_group,0-9,10-19,20-29,30-39,40-49,50-59
title,,,,,,
Copycat (1995),3.500000,3.425000,3.255682,3.397059,3.409091,3.538462
"Cry, the Beloved Country (1995)",0.000000,4.500000,4.090909,3.250000,3.750000,3.857143
Cutthroat Island (1995),1.800000,2.370370,2.172414,2.703704,3.142857,2.866667
Dangerous Minds (1995),2.666667,2.954545,3.233333,3.142857,3.000000,3.222222
Dead Man Walking (1995),4.454545,3.975460,3.898123,4.011628,3.892857,4.040000
Dead Presidents (1995),3.500000,2.962264,2.828571,2.939394,2.785714,2.642857
Dracula: Dead and Loving It (1995),3.600000,2.440000,1.846154,2.681818,2.666667,2.411765
Father of the Bride Part II (1995),3.400000,2.861538,2.940171,3.155172,3.000000,3.130435
Four Rooms (1995),2.333333,3.560000,3.370370,3.000000,3.000000,2.400000
